In [3]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import json

In [4]:
consumer_key='DsqP9rQPObMhkGXo7MXlyxygR'
consumer_secret='CjP7jOIM2iKuCdbhbUJEZBrB9VmOKMJnt9pnfechdqcyGyfQM9'
access_token ='3274806836-CCeR9sSXEgCB6V88ww8kW6pQJoYesx96ip35Dqk'
access_secret='89N3PDyZzakoH7W6n8ZrjGDDktjh8iWFG6eKRvi3kvpQ'

In [10]:
class TweetListener(tweepy.Stream):

    def __init__(self, csocket):
        self.client.socket = csocket

    def on_data(self, data):
        try:
            msg = json.loads( data )
            print( msg['text'].encode('utf-8') )
            self.client_socket.send( msg['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))

        return True

    def on_error(self, status):
          print(status)
          return True

    def on_status(self, status):
        print(status.text)

In [11]:
def sendData(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)

  twitter_stream = Stream(auth, TweetListener(c_socket))
  twitter_stream.filter(track=['soccer'])

In [ ]:
if __name__ == "__main__":
  s = socket.socket()         # Create a socket object
  host = "127.0.0.1"     # Get local machine name
  port = 5555                 # Reserve a port for your service.
  s.bind((host, port))        # Bind to the port

  print("Listening on port: %s" % str(port))

  s.listen(5)                 # Now wait for client connection.
  c, addr = s.accept()        # Establish connection with client.

  print( "Received request from: " + str( addr ) )

  sendData( c )

Listening on port: 5555


In [1]:

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
sc = SparkContext()

In [3]:
ssc = StreamingContext(sc, 10)

In [4]:
sqlContecxt = SQLContext(sc)

In [5]:
socket_stream = ssc.socketTextStream("127.0.0.1", 9998)

In [6]:
lines = socket_stream.window(20)

In [7]:
from collections import namedtuple
fields = ("tag", "count")
Tweet = namedtuple('Teweet', fields)

In [8]:
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) )

In [9]:
ssc.start()